<a href="https://colab.research.google.com/github/SHermukhammadjon/Mohirdev_Data_Science_praktikum/blob/main/ML/scikit_learn3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Maschine Learning Loyiha qadamalri
(3-notebook)

<img src="https://drive.google.com/uc?id=1ZKgMJ92OMgfBHsvQmFmNhXw5aM9P9vXu">

In [64]:
import pandas as pd
import numpy as np 

## 3-QADAM. MLga tayyorgarlik. Pipeline. Sonlar
### 25.05.2023

In [89]:
from sklearn.model_selection import train_test_split

dataset = pd.read_csv("https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true")
dataset

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [66]:
from sklearn.base import BaseEstimator, TransformerMixin
# we should columns indexs 
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombineAttributesAdder(BaseEstimator, TransformerMixin):
  def __init__(self, add_bedrooms_per_room = False):
    self.add_bedrooms_per_room = add_bedrooms_per_room
  
  def fit(self, X, y = None):
    return self
  
  def transform(self, X):
    rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
    population_per_household = X[:, population_ix] / X[:, households_ix]
    if self.add_bedrooms_per_room:
      bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
      return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
    else:
      return np.c_[X, rooms_per_household, population_per_household]

In [113]:
attributadder = CombineAttributesAdder(add_bedrooms_per_room = True)
attributadder.transform(dataset.values)

array([[-122.23, 37.88, 41.0, ..., 6.984126984126984, 2.5555555555555554,
        0.14659090909090908],
       [-122.22, 37.86, 21.0, ..., 6.238137082601054, 2.109841827768014,
        0.15579659106916466],
       [-122.24, 37.85, 52.0, ..., 8.288135593220339, 2.8022598870056497,
        0.12951601908657123],
       ...,
       [-121.22, 39.43, 17.0, ..., 5.20554272517321, 2.325635103926097,
        0.21517302573203195],
       [-121.32, 39.43, 18.0, ..., 5.329512893982808, 2.1232091690544412,
        0.21989247311827956],
       [-121.24, 39.37, 16.0, ..., 5.254716981132075, 2.616981132075472,
        0.22118491921005387]], dtype=object)

In [114]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_pipline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('atribut_adder', CombineAttributesAdder(add_bedrooms_per_room = True)),
    ('standart_scaler', StandardScaler())
])

num_pipline.fit_transform(dataset.drop('ocean_proximity', axis = 1).values)

array([[-1.32783522,  1.05254828,  0.98214266, ...,  0.62855945,
        -0.04959654, -1.02998783],
       [-1.32284391,  1.04318455, -0.60701891, ...,  0.32704136,
        -0.09251223, -0.8888972 ],
       [-1.33282653,  1.03850269,  1.85618152, ...,  1.15562047,
        -0.02584253, -1.29168566],
       ...,
       [-0.8237132 ,  1.77823747, -0.92485123, ..., -0.09031802,
        -0.0717345 ,  0.02113407],
       [-0.87362627,  1.77823747, -0.84539315, ..., -0.04021111,
        -0.09122515,  0.09346655],
       [-0.83369581,  1.75014627, -1.00430931, ..., -0.07044252,
        -0.04368215,  0.11327519]])

In [115]:
from sklearn.compose import ColumnTransformer

cat_attribs = ['ocean_proximity'] 
columns_num = list(dataset.drop('ocean_proximity', axis = 1))

ful_pipline = ColumnTransformer([
    ('num', num_pipline, columns_num),
    ('cat', OneHotEncoder(), cat_attribs)
])

ful_pipline.fit_transform(dataset)[0:5, ]

array([[-1.32783522,  1.05254828,  0.98214266, -0.8048191 , -0.97247648,
        -0.9744286 , -0.97703285,  2.34476576,  2.12963148,  0.62855945,
        -0.04959654, -1.02998783,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ],
       [-1.32284391,  1.04318455, -0.60701891,  2.0458901 ,  1.35714343,
         0.86143887,  1.66996103,  2.33223796,  1.31415614,  0.32704136,
        -0.09251223, -0.8888972 ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ],
       [-1.33282653,  1.03850269,  1.85618152, -0.53574589, -0.82702426,
        -0.82077735, -0.84363692,  1.7826994 ,  1.25869341,  1.15562047,
        -0.02584253, -1.29168566,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ],
       [-1.33781784,  1.03850269,  1.85618152, -0.62421459, -0.71972345,
        -0.76602806, -0.73378144,  0.93296751,  1.16510007,  0.15696608,
        -0.0503293 , -0.4496128 ,  0.        ,  0.        ,  0.        ,
         1.        

In [136]:
columns = list(dataset)
for new_column in ['rooms_per_household', 'population_per_household', 'bedrooms_per_room']:
  columns.append(new_column)
for new_column in list(dataset['ocean_proximity'].unique()):
  columns.append(new_column)
columns

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'ocean_proximity',
 'rooms_per_household',
 'population_per_household',
 'bedrooms_per_room',
 'NEAR BAY',
 '<1H OCEAN',
 'INLAND',
 'NEAR OCEAN',
 'ISLAND']

In [133]:
list(dataset['ocean_proximity'].unique())

['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND']

In [134]:
dataset_copy = pd.DataFrame(data = ful_pipline.fit_transform(dataset.copy()), columns = columns)
dataset_copy

ValueError: ignored

# Split the data
<img src='https://cdn-coiao.nitrocdn.com/CYHudqJZsSxQpAPzLkHFOkuzFKDpEHGF/assets/static/optimized/rev-85bf93c/wp-content/uploads/2022/05/train-test-split_simple-visual-example_v2.png'>

In [112]:



# X, Y = dataset.drop('median_house_value', axis = 1),  dataset['median_house_value']
# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 31)

# 4-QADAM. Machine Learning
###  25.05.2023

In [70]:
from sklearn.linear_model import LinearRegression 

In [76]:
model = LinearRegression()
x_prepared = ful_pipline.fit_transform(housing)
y = housing_lables

In [77]:
model.fit(x_prepared, y)

LinearRegression()